In [2]:
"""
Relevance Detection
"""

'\nRelevance Detection\n'

In [94]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import utils
import importlib
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import Counter
from sklearn.metrics import confusion_matrix
import score
from sklearn.preprocessing import StandardScaler

In [125]:
#run this cell to reload the preprocessing module
importlib.reload(preprocessing)
importlib.reload(utils)

<module 'utils' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/utils.py'>

In [96]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [97]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [98]:
stances_tr, stances_val = preprocessing.train_test_split(train_bodies, train_stances)
stances_tr.shape, stances_val.shape

((39892, 3), (10080, 3))

In [99]:
# this one takes a while!
idf = preprocessing.build_idf(train_bodies, stances_tr)

In [102]:
#this is just a comparison between using IDF score and not using IDF score - not related to the model
#change the body id to see
body = preprocessing.get_body(5, train_bodies)
#no IDF
processed2 = preprocessing.process_body(body)
print(processed2['common_nouns'],processed2['common_verbs'])

#with IDF
processed = preprocessing.process_body(body, idf)
print(processed['common_nouns'],processed['common_verbs'])

['burger', 'year', 'friend', 'news', 'report'] ['bought', 'showed', 'started', 'wonder', 'went']
['burger', 'year', 'friend', 'charity', 'mcjordan'] ['bought', 'started', 'showed', 'dissuaded', 'sauce']


In [103]:
body = preprocessing.get_body(1369, train_bodies)
processed = preprocessing.process_body(body, idf)
print(processed['first_sentence']['tokens'])
print(processed['significant_sentence']['tokens'])
print(processed['first_sentence']['adverbs'],processed['significant_sentence']['adverbs'])
print(processed['first_sentence']['adjectives'],processed['significant_sentence']['adjectives'])
print(processed['first_sentence']['verbs'],processed['significant_sentence']['verbs'])

['unconfirmed', 'report', 'circulating', 'social', 'medium', 'islamic', 'state', 'group', 'carried', 'chemical', 'attack', 'battling', 'kurdish', 'force', 'kobani']
['kurdish', 'affair', 'analyst', 'mutlu', 'civiroglu', 'spoke', 'remaining', 'doctor', 'inside', 'kobani', 'told', 'victim', 'civilian']
[] []
['unconfirmed', 'social', 'islamic', 'chemical', 'kurdish'] ['kurdish', 'civiroglu']
['circulating', 'carried', 'battling'] ['spoke', 'remaining', 'told']


In [126]:
#this takes a while!
body_info = preprocessing.process_bodies(train_bodies, idf)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
done! processed 1683


In [127]:
feats_list = [
    'shared_nouns',
    'shared_verbs',
    'shared_bigrams',
    'shared_trigrams',
    'shared_tokens',

    'shared_nouns_fst',
    'shared_verbs_fst',
    'shared_bigrams_fst',
    'shared_trigrams_fst',
    'shared_tokens_fst',

    'shared_nouns_sig',
    'shared_verbs_sig',
    'shared_bigrams_sig',
    'shared_trigrams_sig',
    'shared_tokens_sig',

    
    'cos_nouns_sig',
    'cos_bigrams_sig',
    'cos_trigrams_sig',
    'cos_tokens_sig',

    'cos_nouns_fst',
    'cos_bigrams_fst',
    'cos_trigrams_fst',
    'cos_tokens_fst',
    
#     'sentiment_pos',
#     'sentiment_neg',
#     'sentiment_neu',
#     'sentiment_compound',
#     'sentiment_pos_fst',
#     'sentiment_neg_fst',
#     'sentiment_neu_fst',
#     'sentiment_compound_fst',
#     'sentiment_pos_sig',
#     'sentiment_neg_sig',
#     'sentiment_neu_sig',
#     'sentiment_compound_sig',
]

In [136]:
# this one takes a while also!

#train data
data_feats = [preprocessing.get_feats(i, body_info) for i in stances_tr.values]
train_df = pd.DataFrame()
for i in feats_list:
    train_df[i] = [x[i] for x in data_feats]

#val data
val_feats = [preprocessing.get_feats(i, body_info) for i in stances_val.values]
val_df = pd.DataFrame()
for i in feats_list:
    val_df[i] = [x[i] for x in val_feats]

In [137]:
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_trigrams_fst,shared_tokens_fst,...,sentiment_neu,sentiment_compound,sentiment_pos_fst,sentiment_neg_fst,sentiment_neu_fst,sentiment_compound_fst,sentiment_pos_sig,sentiment_neg_sig,sentiment_neu_sig,sentiment_compound_sig
0,0,0,0,0,0,0,0,0,0,0,...,-0.125444,-0.8167,-0.098,0.194,-0.096,-0.8167,-0.130,0.194,-0.064,-0.8167
1,2,0,2,1,6,1,0,0,0,3,...,0.115000,0.3182,-0.038,-0.077,0.115,0.3182,-0.178,0.000,0.178,-0.0772
2,0,0,0,0,0,0,0,0,0,0,...,0.152800,0.0000,0.000,0.000,0.000,0.0000,0.000,-0.231,0.231,0.2732
3,0,0,0,0,0,0,0,0,0,0,...,0.098000,0.8300,0.000,-0.186,0.186,0.8300,0.000,0.000,0.000,0.0000
4,1,0,1,0,3,0,0,0,0,0,...,0.100029,0.4939,0.000,-0.198,0.198,0.4939,-0.385,0.000,0.385,-0.6124


In [138]:
train_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_tr['Stance'])]
val_df['label'] = [0 if x == "unrelated" else 1 for x in list(stances_val['Stance'])]
train_df.head()

,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,shared_nouns_fst,shared_verbs_fst,shared_bigrams_fst,shared_trigrams_fst,shared_tokens_fst,...,sentiment_compound,sentiment_pos_fst,sentiment_neg_fst,sentiment_neu_fst,sentiment_compound_fst,sentiment_pos_sig,sentiment_neg_sig,sentiment_neu_sig,sentiment_compound_sig,label
0,0,0,0,0,0,0,0,0,0,0,...,-0.8167,-0.098,0.194,-0.096,-0.8167,-0.130,0.194,-0.064,-0.8167,0
1,2,0,2,1,6,1,0,0,0,3,...,0.3182,-0.038,-0.077,0.115,0.3182,-0.178,0.000,0.178,-0.0772,1
2,0,0,0,0,0,0,0,0,0,0,...,0.0000,0.000,0.000,0.000,0.0000,0.000,-0.231,0.231,0.2732,0
3,0,0,0,0,0,0,0,0,0,0,...,0.8300,0.000,-0.186,0.186,0.8300,0.000,0.000,0.000,0.0000,0
4,1,0,1,0,3,0,0,0,0,0,...,0.4939,0.000,-0.198,0.198,0.4939,-0.385,0.000,0.385,-0.6124,1


In [116]:
scaler = StandardScaler()

for i in feats_list:
    train_df[i] = scaler.fit_transform(train_df[i].values.reshape(-1,1))
    val_df[i] = scaler.fit_transform(val_df[i].values.reshape(-1,1))
train_df.head()

/Users/dannyyang/Library/Python/3.6/lib/python/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,shared_nouns,shared_verbs,shared_bigrams,shared_trigrams,shared_tokens,sentiment_pos,sentiment_neg,sentiment_neu,sentiment_compound,shared_nouns_fst,...,sentiment_compound_sig,cos_nouns_sig,cos_bigrams_sig,cos_trigrams_sig,cos_tokens_sig,cos_nouns_fst,cos_bigrams_fst,cos_trigrams_fst,cos_tokens_fst,label
0,-0.398931,-0.208022,-0.420621,-0.218715,-0.728784,-0.490928,0.717276,-0.412417,-1.517528,-0.487906,...,-1.227446,0.338732,0.223621,0.081789,0.442448,0.427123,0.228363,0.164608,0.528702,0
1,2.781041,-0.208022,1.473325,1.688798,1.679355,-0.187008,-0.859125,0.888333,0.622570,0.517754,...,0.240603,-1.114258,0.223621,0.081789,-2.160286,-0.257939,0.228363,0.164608,-1.101853,1
2,-0.398931,-0.208022,-0.420621,-0.218715,-0.728784,-0.399542,-0.960883,1.092822,0.022535,-0.487906,...,0.936214,0.338732,0.223621,0.081789,0.442448,0.427123,0.228363,0.164608,0.528702,0
3,-0.398931,-0.208022,-0.420621,-0.218715,-0.728784,0.174325,-0.964392,0.796366,1.587678,-0.487906,...,0.393860,0.338732,0.223621,0.081789,0.442448,0.427123,0.228363,0.164608,0.528702,0
4,1.191055,-0.208022,0.526352,-0.218715,0.475285,-0.539235,-0.573401,0.807341,0.953890,-0.487906,...,-0.821872,-1.827257,0.223621,0.081789,-0.997901,0.427123,0.228363,0.164608,0.528702,1


In [139]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso
from sklearn.ensemble import RandomForestClassifier
#adjust params as you see fit
model = RandomForestClassifier(n_estimators = 250, min_samples_split = 10, min_samples_leaf = 5, max_depth = 10)
model2 = LogisticRegression()
# Lasso(alpha=0.01)
# train_df.iloc[:,-1].values.reshape(-1)
model.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])
model2.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [140]:
tr_acc = model.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

tr_acc = model2.score(train_df.iloc[:,:-1], train_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% training accuracy'.format(tr_acc*100))
val_acc = model2.score(val_df.iloc[:,:-1], val_df.iloc[:,-1].values.reshape(-1))
print('{0:.2f}% validation accuracy'.format(val_acc*100))

97.30% training accuracy
96.03% validation accuracy
95.99% training accuracy
95.60% validation accuracy


In [141]:
# get coefficients - logistic
[(feats_list[i],model2.coef_[0][i]) for i in list(range(len(feats_list)))]

#coefficients - lasso
#model.coef_

[('shared_nouns', 0.98083729928844832),
 ('shared_verbs', 0.51880282645235487),
 ('shared_bigrams', 0.88605879940799481),
 ('shared_trigrams', -1.5563368453065831),
 ('shared_tokens', 1.0300675058184925),
 ('shared_nouns_fst', -0.42186185828022588),
 ('shared_verbs_fst', -0.22982513905878918),
 ('shared_bigrams_fst', -1.5151274586331087),
 ('shared_trigrams_fst', 0.72991329911989444),
 ('shared_tokens_fst', 1.4394747490206619),
 ('shared_nouns_sig', -0.16772889907350816),
 ('shared_verbs_sig', -0.42444987047972144),
 ('shared_bigrams_sig', -1.1984177060204881),
 ('shared_trigrams_sig', 1.393138298156823),
 ('shared_tokens_sig', 0.16903452262207572),
 ('cos_nouns_sig', -0.054904773909058777),
 ('cos_bigrams_sig', 2.3311640356425358),
 ('cos_trigrams_sig', -1.522093385962541),
 ('cos_tokens_sig', -6.1491637578250673),
 ('cos_nouns_fst', -0.45162223269760338),
 ('cos_bigrams_fst', -0.45146599782992974),
 ('cos_trigrams_fst', -0.32163273318470387),
 ('cos_tokens_fst', -1.5265626736916835),

In [142]:
# #usage example for json dump
utils.rf_json_dump(model, list(train_df.iloc[:,:-1]), "test_rf_dump.json")

In [143]:
true_label = val_df.iloc[:,-1]
prediction = model.predict(val_df.iloc[:,:-1])
matrix = confusion_matrix(true_label,prediction)
print('confusion matrix: \n{}\n'.format(matrix))
tn1, fp1, fn1, tp1 = matrix.ravel()

confusion matrix: 
[[7096  197]
 [ 203 2584]]



In [144]:
#use FNC scorer to generate score report
label_prediction = ["discuss" if i == 1 else "unrelated" for i in prediction]
label_actual = pd.DataFrame(stances_val)['Stance']
score.report_score(label_actual, label_prediction)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |     0     |     0     |    773    |    37     |
-------------------------------------------------------------
| disagree  |     0     |     0     |    146    |    21     |
-------------------------------------------------------------
|  discuss  |     0     |     0     |   1665    |    145    |
-------------------------------------------------------------
| unrelated |     0     |     0     |    197    |   7096    |
-------------------------------------------------------------
Score: 3668.75 out of 4610.25	(79.5781139851418%)


79.5781139851418